<a href="https://colab.research.google.com/github/Falconwatch/cybersec_ht/blob/main/Implementation/Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import lightgbm as lgbm
import joblib
from tqdm import tqdm
import pandas as pd
import rapidfuzz

In [2]:
examples = pd.read_excel("examples.xlsx")
cust_1 = pd.read_excel("cust_1.xlsx")
cust_2 = pd.read_excel("cust_2.xlsx")

In [3]:
class HotelComparator():
  def __init__(self, path):
    """
    param path: Путь до сохранённой модели
    """
    self.__clf = joblib.load('hotel_clf.pkl')
    self.__feature_fields = ['hotel_name', 'city_name', 'hotel_address', 'star_rating', 'postal_code']
    self.__th = 0.0002

  def similarity_ratio_func(self, df, field):
    def string_similarity(s1,s2):
      """Вычисляет сходство строк
      param s1: Данные об отеле по версии первого поставщика
      param s2: Данные об отеле по версии второго поставщика
      return: Степень сходства между данными
      """
      return rapidfuzz.fuzz_cpp.WRatio(s1,s2)/100

    def postalcode_similarity(s1, s2):
      """Вычисляет сходство почтовых индексов
      param s1: Индекс по версии первого поставщика
      param s2: Индекс по версии второго поставщика
      return: Степень сходства между Индексами
      """
      longest_string = max([len(s1), len(s2)])
      s1 = s1+"0"*(longest_string - len(s1))
      s2 = s2+"0"*(longest_string - len(s2))
      t = [a==b for a,b in zip(s1, s2)][::-1]
      max_possible_similarity = sum([10**i for i in range(longest_string)])
      current_similarity = sum([t[i]*10**i for i in range(len(t))])
      difference = max_possible_similarity - current_similarity
      difference_ratio = difference/max_possible_similarity
      return 1-difference_ratio
    
    def rating_star_diff(s1, s2):
      """Вычисляет различие рейтингов (количество звёзд)
      param s1: Количество звёзд по версии первого поставщика
      param s2: Количество звёзд по версии второго поставщика
      return: Степень различия между рейтингами
      """
      try:
        r = abs(float(s1)- float(s2))
        return r
      except:
        return None

    def universal_wrapper(x1, x2, func):
      """Обёртка над вычислениями сходства/различия. Решает проблему нанов и приведения типов
      param s1: Индекс по версии первого поставщика
      param s2: Индекс по версии второго поставщика
      return: Степень сходства между Индексами
      """
      if pd.isna(x1) or pd.isna(x2):
        return None
      x1=str(x1).upper()
      x2=str(x2).upper()
      return func(x1,x2)

    name1 = "c1." + field
    name2 = "c2." + field
    name3 = field + "_similarity"
    
    if "postal_code" in field:
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], postalcode_similarity),
                                            axis=1)
    elif "star_rating" in field:
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], rating_star_diff),
                                            axis=1)
    else:
      df[name3] = df[[name1, name2]].apply(lambda x: universal_wrapper(x[name1], x[name2], string_similarity),
                                            axis=1)

  def generate_features(self, df):
    """ Генерирует фичи сходства
    param df: pandas dataframe с сопоставленными списками двух поставщиков
    """
    for ff in tqdm(self.__feature_fields):
      self.similarity_ratio_func(df, ff)


  def compare_hotels_lists(self, hotels_1, hotels_2,
                           country_code_name_1 = "c1.country_code",
                           country_code_name_2 = "c2.country_code",):
    """Принимает на вход списки отелей от двух поставщиков. Возращает список мэтчей
    param hotels_1: pandas dataframe со списком отелей от поставщика 1
    param hotels_2: pandas dataframe со списком отелей от поставщика 2
    return: pands dataframe с итоговым списком мэтчей
    """
    hotels_1 = hotels_1.rename({country_code_name_1:"country_code"}, axis=1)
    hotels_2 = hotels_2.rename({country_code_name_2:"country_code"}, axis=1)
    countries = hotels_1["country_code"].unique()

    #Собираем полное пересечение (с учётом, что страны раскиданы верно)
    country_datas = list()
    for country in countries:
      h1 = hotels_1[hotels_1["country_code"] == country]
      h2 = hotels_2[hotels_2["country_code"] == country]
      total = h1.reset_index(drop=True).merge(h2.reset_index(drop=True),
                                              on =["country_code"])
      country_datas.append(total)
    full_data = pd.concat(country_datas)

    # Добавляем фичи сходства
    self.generate_features(full_data)

    #Применяем модель
    features = [f+"_similarity" for f in self.__feature_fields]
    probas = self.__clf.predict_proba(full_data[features])[:,1]
    predictions = (probas > self.__th).astype(int)

    full_data["match"] = predictions
    result = full_data.loc[full_data["match"]==1, ["c1.key", "c2.key"]]

    return result


In [6]:
hc = HotelComparator("hotel_clf.pkl")
result = hc.compare_hotels_lists(cust_1, cust_2)

100%|██████████| 5/5 [09:51<00:00, 118.22s/it]


In [8]:
result.reset_index(drop=True).to_csv("submit.csv")